### ANALYSIS OF MOVIE REVIEW From IMDB
##### USING  Classical Machine Learning Models and Ensemble of Classical Machine Learning Models 

<h3> 
     Author: Sushma G.
</h3>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import cross_val_score
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr
from scipy import stats
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler

from IPython import get_ipython
get_ipython().run_line_magic('matplotlib', 'inline')


from pandas import ExcelWriter
from math import sqrt
from scipy.stats import norm, skew
from scipy.special import boxcox1p

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
plt.style.use('bmh')


## Setting plotting configurations
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

pd.options.display.float_format = '{:.2f}'.format
import numpy as np
import matplotlib.pyplot as plt
#plt.style.context('seaborn-talk')
plt.style.use(['tableau-colorblind10'])
#plt.style.use('fivethirtyeight')
params = {'legend.fontsize': '16',
          'figure.figsize': (15, 5),
         'axes.labelsize': '20',
         'axes.titlesize':'30',
         'xtick.labelsize':'18',
         'ytick.labelsize':'18'}
plt.rcParams.update(params)

plt.rcParams['text.color'] = '#A04000'
plt.rcParams['xtick.color'] = '#283747'
plt.rcParams['ytick.color'] = '#808000'
plt.rcParams['axes.labelcolor'] = '#283747'

## Task1: Load Data

In [ ]:
raw_reviews_df = pd.read_csv("../input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv") #read from csv file

In [ ]:
raw_reviews_df.shape #view dimensions of the pandas daframe

In [ ]:
raw_reviews_df.iloc[0:1] #view one movie review

In [ ]:
raw_reviews_df.info() #vew information abot Daframe in general

## Task2: Preprocess Data

In this task, data preprocessing tasks such as adjust class imbalances , filling out missing values are done. This is done to prevent bias later for Classification Task

In [ ]:
plt.figure(figsize = (17,9))
raw_reviews_df["sentiment"].value_counts().plot(kind = 'barh')
plt.title("Distribution of Classes in the Reviews")
plt.ylabel("Sentiment")
plt.xlabel("Number of samples")
plt.show();

In [ ]:
print("Total Samples for Positive Sentiments: ",len(raw_reviews_df[raw_reviews_df["sentiment"]=="positive"]))
print("Total Samples for Negative Sentiments: ",len(raw_reviews_df[raw_reviews_df["sentiment"]=="negative"]))

Looks like the classes(sentiments) are balanced perfectly. SO need for sampling. Now check if there are any missing values in review column.

In [ ]:
print("Total Samples which do not have reviews: ",len(raw_reviews_df[raw_reviews_df["review"]==None]))

Awesome, no missing reviews so not need for data imputation. Now lets view frequenly used words in reviews in a word cloud

In [ ]:
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from wordcloud import WordCloud #library that allows word cloud to be generated

def generate_word_cloud(dataframe_df,column_name,img_file=None,background_color="black",max_words=2000):
    """ This function generates word cloud
    
    Args:
        dataframe: the datafram object which contains column whose word cloud is to be generated
        column_name: the name of the column in dataframw whose cloud is to be generated
        
    Returns:
        None
    """
    #convert the column specified by column_name in dataset_df(dataframe) to list
    sentences_list = dataframe_df[column_name].tolist()
    
    #convert a list to sentence
    #sentences_single_string = " ".join(sentences_list)
    sentences_single_string = " ".join(map(str,sentences_list))
    
    #plot word cloud
    plt.figure(figsize=(15,15))
    
    img_mask = None
    
    if img_file != None:
        img_mask = np.array(Image.open(img_file))

    wc = WordCloud(mask=img_mask,
                   max_words=max_words,
                   background_color=background_color)        
        
    plt.imshow(wc.generate(sentences_single_string),interpolation="bilinear")
    plt.title("Word Cloud For " +  column_name)
    plt.show()

Lets Plot word cloud for all the reviews

In [ ]:
import time
tic= time.time()
generate_word_cloud(raw_reviews_df,"review",background_color="white")
toc = time.time()
diff = 1000*(toc - tic)
print("Total Time Taken: " + str(diff) + " ms")

### Task 3: Text Preprocessing

In [ ]:
#import required libraries
from nltk.corpus import stopwords #for stopwords
from nltk.stem import PorterStemmer #for word stemming
from nltk.tokenize import TweetTokenizer #for toekinizing string to list of words
import string #for punctuation
import re #for regular expression
import numpy as np

In [ ]:
def process_string(text):
    """This function returns a processed list of words from the given text
    
    This function removes html elements and urls using regular expression, then
    converts string to list of workds, them find the stem of words in the list of words and
    finally removes stopwords and punctuation marks from list of words.
    
    Args:
        text(string): The text from which hrml elements, urls, stopwords, punctuation are removed and stemmed
        
    Returns:
        clean_text(string): A text formed after text preprocessing.
    """
    
    #remove any urls from the text
    text = re.sub(r"https:\/\/.*[\r\n]*","",text)
    
    #remove any urls starting from www. in the text
    text = re.sub(r"www\.\w*\.\w\w\w","",text)
    
    #remove any html elements from the text
    text = re.sub(r"<[\w]*[\s]*/>","",text)
    
    #remove prediods  marks
    text = re.sub(r"[\.]*","",text)
    
    #initilze tweet tokenizer 
    tokenizer = TweetTokenizer(preserve_case=False,strip_handles=True,reduce_len=True)
    
    #tokenize text
    text_tokens = tokenizer.tokenize(text)
    
    #intizlize porter stemmer
    porter_stemmer = PorterStemmer()
    
    #get english stopwords
    english_stopwords = stopwords.words("english")
    
    cleaned_text_tokens = [] # a list to hold cleaned text tokens
    
    for word in text_tokens:
        if((word not in english_stopwords) and #remove stopwords
            (word not in string.punctuation)): #remove punctuation marks
                
                stemmed_word = porter_stemmer.stem(word) #get stem of the current word
                cleaned_text_tokens.append(stemmed_word) #appened stemmed word to list of cleaned list
    
    #combine list into single string
    clean_text = " ".join(cleaned_text_tokens)
    
    return clean_text

In [ ]:
raw_reviews_df["review"].iloc[0]

In [ ]:
process_string(raw_reviews_df["review"].iloc[0])

<p> <h5> clearly this cleaned review is free of html tags as well now. Lets apply this new text cleaning method to all 50k reviews </h5> </p>

In [ ]:
raw_reviews_df["review"] = raw_reviews_df["review"].apply(process_string)

In [ ]:
#save processed reviews for future use
#save file to csv
raw_reviews_df.to_csv("imdb_reviews_cleaned.csv")

In [ ]:
raw_reviews_df = pd.read_csv("imdb_reviews_cleaned.csv")

In [ ]:
raw_reviews_df.info()

In [ ]:
raw_reviews_df["review"].iloc[0]

<p> <h5>
Lets look at reviews after they have been cleaned
</h5> </p>

In [ ]:
tic = time.time()
generate_word_cloud(raw_reviews_df,"review",background_color="white")
toc = time.time()
diff = toc - tic
print("Total Time Taken: " + str(diff) + " ms")

<p> <h5> 
Clearly the new text preprocessing method resulted in much more cleaner reviews, the prominent "br" tag that has massive occurence is not visible anymore. Thanks to regular epression. Lets further examine these word could by looking into positive only and negative only reviews </h5> </p>

<p> <h5> The review have been saved and loaded.</h5> </p>

### Task 4: Split Data into Train and Test Set

<p> <h5>
The data set has to split into train(80%)/valid(10%)/test(10%) set. Train set will be used to train machine learning models and validation set will be used to tune the machine learning models and the test will be used to evaluate the performance of ml models. <br> <br>
There are two classes ("positive" and "negative") in the dataset. A blanced dataset containing equal number of samples from both classes.
</h5> </p>

In [ ]:
#scikit learn library that splits data
from sklearn.model_selection import train_test_split

In [ ]:
#first spli data into train and test set
review_train, review_test, labels_train, labels_test = train_test_split(raw_reviews_df["review"], raw_reviews_df["sentiment"], test_size=0.1, random_state=0)

In [ ]:
#second split train set into train and valid set
review_train, review_valid, labels_train, labels_valid = train_test_split(review_train, labels_train, test_size=0.1111, random_state=0)

In [ ]:
#see the sample count in all sets
print("Train Set: ")
print("Positive Samples count: " + str(len(review_train[labels_train=="positive"] )  ) )
print("Negative Samples count: " + str(len(review_train[labels_train=="negative"] )  ) )
print("Total Samples count: " + str(len(review_train)  ) )

In [ ]:
#see the sample count in all sets
print("Valid Set: ")
print("Positive Samples count: " + str(len(review_valid[labels_valid=="positive"] )  ) )
print("Negative Samples count: " + str(len(review_valid[labels_valid=="negative"] )  ) )
print("Total Samples count: " + str(len(review_valid)  ) )

In [ ]:
#see the sample count in all sets
print("Test Set: ")
print("Positive Samples count: " + str(len(review_test[labels_test =="positive"] )  ) )
print("Negative Samples count: " + str(len(review_test[labels_test =="negative"] )  ) )
print("Total Samples count: " + str(len(review_test) ))

In [ ]:
#convert train matrix from numpy matrix to pandas
review_train_df = pd.DataFrame()
review_train_df["review"] = review_train
review_train_df["sentiment"] = labels_train
review_train_df.info()

In [ ]:
#convert valid matrix from numpy matrix to pandas
review_valid_df = pd.DataFrame()
review_valid_df["review"] = review_valid
review_valid_df["sentiment"] = labels_valid
review_valid_df.info()

In [ ]:
#convert test matrix from numpy matrix to pandas
review_test_df = pd.DataFrame()
review_test_df["review"] = review_test
review_test_df["sentiment"] = labels_test
review_valid_df.info()

In [ ]:
#make sure the reviews are in string format
review_train_df["review"] = review_train_df["review"].apply(lambda row_text: str(row_text))
review_valid_df["review"] = review_valid_df["review"].apply(lambda row_text: str(row_text))
review_test_df["review"] = review_test_df["review"].apply(lambda row_text: str(row_text))

### Task 5: Generate Features from reviews And Numerical labels for Sentiment

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer() #instantiate vectorizer

In [ ]:
feature_matrix_train = vectorizer.fit_transform(review_train_df["review"].tolist())

In [ ]:
feature_matrix_train.shape

In [ ]:
feature_matrix_valid = vectorizer.transform(review_valid_df["review"].tolist())

In [ ]:
feature_matrix_valid.shape

In [ ]:
feature_matrix_test = vectorizer.transform(review_test_df["review"].tolist())

In [ ]:
feature_matrix_test.shape

<p> <h5>
The sentiment for each review is labelled as positive or negative. Machine Learning Models require Numberical values so lets convert these labels to numerical labels
</h5> </p>

In [ ]:
#import label encoder
from sklearn import preprocessing #library that alllows label encoding
label_encoder = preprocessing.LabelEncoder() #instantiate label encoder

In [ ]:
label_matrix_train = label_encoder.fit_transform(review_train_df["sentiment"]) #encoded

In [ ]:
label_matrix_train.shape

In [ ]:
label_matrix_valid = label_encoder.fit_transform(review_valid_df["sentiment"]) #encoded

In [ ]:
label_matrix_valid.shape

In [ ]:
label_matrix_test = label_encoder.fit_transform(review_test_df["sentiment"]) #encoded

In [ ]:
label_matrix_test.shape

<p><h5> 
Term-Doument Matrix is a poular matrix design in NLP. Lets create a term document matrix
</h5></p>

In [ ]:
dt_train_df = pd.DataFrame.sparse.from_spmatrix(data = feature_matrix_train, columns = vectorizer.get_feature_names())
dt_valid_df = pd.DataFrame.sparse.from_spmatrix(data = feature_matrix_valid, columns = vectorizer.get_feature_names())
dt_test_df = pd.DataFrame.sparse.from_spmatrix(data = feature_matrix_test, columns = vectorizer.get_feature_names())

<p> <h5> The plot above shows frequnecy some top words used in the reviews. Now lets crea pipeline to do this task </h5> </p>

In [ ]:
from matplotlib import pyplot as plt
import time

def plot_top_words(document_term_df,upper_threshold=10000,lower_threshold=10,count=20):
    
    #for time purposes
    tic = time.time()
    
    top_word_freq = {}
    #build frequency dictionary
    for col in document_term_df.columns:
        
        col_count = document_term_df[col].to_numpy().max()
        
        if lower_threshold < col_count < upper_threshold:
            top_word_freq[col] = col_count
            
        if len(top_word_freq) > count:
            break

    #print thresholds
    print("Upper Limit: " + str(upper_threshold))
    print("Lower Limit: " + str(lower_threshold))
    print("Requested Count: " + str(count))
    
    plt.figure(figsize=(20,10))
    
    y_vals = [top_word_freq[word] for word in top_word_freq.keys()]
    
    anotates = list(top_word_freq.keys())
    
    x_vals = [i for i in range(0,len(y_vals))]
    
    plt.scatter(x_vals,y_vals)

    plt.title("Top Word Counts")
    plt.xlabel("Top Words")
    plt.ylabel("Word Frequency")
    
    for y,a,x in zip(y_vals,anotates,x_vals):
        plt.annotate(a, #anotate
                     (x,y), #for this data point 
                    textcoords="offset points", #how to position text
                    xytext=(0,10), #distance from text to points (x,y)
                    ha="center") #horizontal alignment can beleft, right or center 
    
    #for time purposes
    toc = time.time()
    tictoc = (toc-tic)*1000
    print("Time Taken: " + str(tictoc) + " ms")

<p> <h5> Lets Look at the top words used in trainning set </h5> </p>

In [ ]:
plot_top_words(dt_train_df,upper_threshold=10000,lower_threshold=20,count=20)

<p> <h5> Lets Look at the top words used in trainning set whose sentiment is labelled as negative </h5> </p>

In [ ]:
plot_top_words(dt_train_df[label_matrix_train==0],upper_threshold=10000,lower_threshold=10,count=30)

<p> <h5> Lets Look at the top words used in training set whose sentiment is labelled as negative </h5> </p>

In [ ]:
plot_top_words(dt_train_df[label_matrix_train==1],upper_threshold=10000,lower_threshold=10,count=30)

### Task 7: Re-weighing the document term matrix using TF-IDF Scheme

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf_vectorizer = TfidfVectorizer()

In [ ]:
feature_matrix_train = tfidf_vectorizer.fit_transform(review_train_df["review"].tolist())

In [ ]:
feature_matrix_train.shape

In [ ]:
feature_matrix_valid = tfidf_vectorizer.transform(review_valid_df["review"].tolist())

In [ ]:
feature_matrix_valid.shape

In [ ]:
feature_matrix_test = tfidf_vectorizer.transform(review_test_df["review"].tolist())

In [ ]:
feature_matrix_test.shape

In [ ]:
dt_train_df = pd.DataFrame.sparse.from_spmatrix(data = feature_matrix_train, columns = vectorizer.get_feature_names())
dt_valid_df = pd.DataFrame.sparse.from_spmatrix(data = feature_matrix_valid, columns = vectorizer.get_feature_names())
dt_test_df = pd.DataFrame.sparse.from_spmatrix(data = feature_matrix_test, columns = vectorizer.get_feature_names())

### Task 8: Train and Test Machine Learning Models

<p> <h4> Classifer 1:- Naive Bayes </h4> </p>

In [ ]:
tic = time.time()
#Train Naive Bayes Model
#More Details: https://scikit-learn.org/stable/modules/naive_bayes.html
from sklearn.naive_bayes import BernoulliNB #Gaussain assume fatures from normal distribution, Bernouli for boolean, multionomial for discreeet
model_nb = BernoulliNB()
model_nb.fit(feature_matrix_train, label_matrix_train)
toc = time.time()
y_pred_nb = model_nb.predict(feature_matrix_valid)
accuracy_train_set = model_nb.score(feature_matrix_train, label_matrix_train)  #get accuracy on train set
accuracy_valid_set = model_nb.score(feature_matrix_valid, label_matrix_valid)  #get accuracy on valid set
print("Naive Bayes Model, Accuracy (Train Set) : ", accuracy_train_set)
print("Naive Bayes Model, Accuracy (Valid Set) : ", accuracy_valid_set)
print("Time Taken: " + str(((toc-tic)*1000)) + " ms")

<p> <h4> Classifer 2:- Descision Tree </h4> </p>

In [ ]:
tic = time.time()
#Train Descision Trees Model
#More Details: https://scikit-learn.org/stable/modules/classes.html#module-sklearn.tree
from sklearn.tree import DecisionTreeClassifier
maximum_tree_depth= 15
model_dt = DecisionTreeClassifier(max_depth=maximum_tree_depth)
model_dt.fit(feature_matrix_train,label_matrix_train)
toc = time.time()
y_pred_dt = model_nb.predict(feature_matrix_valid)
accuracy_train_set = model_dt.score(feature_matrix_train, label_matrix_train)  #get accuracy on train set
accuracy_valid_set = model_dt.score(feature_matrix_valid, label_matrix_valid)  #get accuracy on valid set
print("Descision Tree Model, Accuracy (Train Set) : ", accuracy_train_set)
print("Descision Tree Model, Accuracy (Valid Set) : ", accuracy_valid_set)
print("Time Taken: " + str(((toc-tic)*1000)) + " ms")

<p> <h4> Classifer 3:- K Nearest Neighbor </h4> </p>

In [ ]:
tic = time.time()
#Train K Nearest Negihbor Model
#More Details: https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm
#MOre Details: https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html
from sklearn.neighbors import KNeighborsClassifier
number_of_neigbors = 3
minkowski_power = 2 # Manhattan Distance = 1, Euclidean Distance = 2
model_knn = KNeighborsClassifier(n_neighbors=number_of_neigbors, p =minkowski_power)
model_knn.fit(feature_matrix_train,  label_matrix_train)
toc = time.time()
y_pred_knn = model_nb.predict(feature_matrix_valid)
accuracy_train_set = model_knn.score(feature_matrix_train, label_matrix_train)  #get accuracy on train set
accuracy_valid_set = model_knn.score(feature_matrix_valid, label_matrix_valid)  #get accuracy on valid set
print("K Nearest Neighbors Model, Accuracy (Train Set) : ", accuracy_train_set)
print("K Nearest Neighbors Model, Accuracy (Valid Set) : ", accuracy_valid_set)
print("Time Taken: " + str(((toc-tic)*1000)) + " ms")

<p> <h4> Classifer 4:- Logistic Regression </h4> </p>

In [ ]:
tic = time.time()
#Train Logistic Regression Model
#More Details: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
from sklearn.linear_model import LogisticRegression #import model for logistic regression
l2_norm = 0.4 # regularization parameter
l2_norm_inverse = 1/l2_norm 
maximum_iterations=4000 #maximum number of iterations
model_lr = LogisticRegression(C=l2_norm_inverse,max_iter=maximum_iterations) #create logistic regression model
model_lr.fit(feature_matrix_train,  label_matrix_train)
toc = time.time()
y_pred_lr = model_nb.predict(feature_matrix_valid)
accuracy_train_set = model_lr.score(feature_matrix_train, label_matrix_train)  #get accuracy on train set
accuracy_valid_set = model_lr.score(feature_matrix_valid, label_matrix_valid)  #get accuracy on valid set
print("Logistic Regression Model, Accuracy (Train Set) : ", accuracy_train_set)
print("Logistic Regression Model, Accuracy (Valid Set) : ", accuracy_valid_set)
print("Time Taken: " + str(((toc-tic)*1000)) + " ms")

<p> <h4> Classifer 5:- Support Vector Machine </h4> </p>

In [ ]:
tic = time.time()
#Train Support Vector Machine Model
#More Details: https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html
from sklearn.svm import LinearSVC
l2_norm = 2.1
l2_norm_inverse = 1/l2_norm
maximum_iterations=4000 #maximum number of iterations
model_svm = LinearSVC(C=l2_norm_inverse,max_iter=maximum_iterations) #create support vector machine model
model_svm.fit(feature_matrix_train,  label_matrix_train)
toc = time.time()
y_pred_svm = model_nb.predict(feature_matrix_valid)
accuracy_train_set = model_svm.score(feature_matrix_train, label_matrix_train)  #get accuracy on train set
accuracy_valid_set = model_svm.score(feature_matrix_valid, label_matrix_valid)  #get accuracy on valid set
print("Support Vector Machine Model, Accuracy (Train Set) : ", accuracy_train_set)
print("Support Vector Machine Model, Accuracy (Valid Set) : ", accuracy_valid_set)
print("Time Taken: " + str(((toc-tic)*1000)) + " ms")

<p> <h4> Classifer 6:- Bagging Classifier (Base Estimator = LinearSVC) </h4> </p>

In [ ]:
tic = time.time()
#Train a MLP classifier
# Details:https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.RadiusNeighborsClassifier.html#sklearn.neighbors.RadiusNeighborsClassifierfrom sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import LinearSVC
l2_norm = 2
l2_norm_inverse = 1/l2_norm
maximum_iterations=4000 #maximum number of iterations
 
model_bc = BaggingClassifier(base_estimator=LinearSVC(C=l2_norm_inverse,max_iter=maximum_iterations),
                             n_estimators=30, 
                             random_state=0)

model_bc.fit(feature_matrix_train,  label_matrix_train)
toc = time.time()
y_pred_stacked = model_bc.predict(feature_matrix_valid)
accuracy_train_set = model_bc.score(feature_matrix_train, label_matrix_train)  #get accuracy on train set
accuracy_valid_set = model_bc.score(feature_matrix_valid, label_matrix_valid)  #get accuracy on valid set
print("Bagging Classifier ML Model, Accuracy (Train Set) : ", accuracy_train_set)
print("Bagging Classifier ML Model, Accuracy (Valid Set) : ", accuracy_valid_set)
print("Time Taken: " + str(((toc-tic)*1000)) + " ms")

<p> 
<h4>
    Classifer 7:- Stacked ML CLassifer (Naive Bayes, K Nearest Neighbor, Support Vector Machine)
</h4>
</p>

In [ ]:
tic = time.time()
#Create a stack of these estimators
estimators = [
    ("nb",model_nb), #stack naive bayes
    ("knn",model_knn), #stack K nearest neighor
    ('svm', model_svm) #stack support vector machine
]

#Train a stacked model
# Details: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html
from sklearn.ensemble import StackingClassifier
model_stacked = StackingClassifier(estimators=estimators, final_estimator=model_lr) #use logisitc regression as the final estimator
model_stacked.fit(feature_matrix_train,  label_matrix_train)
toc = time.time()
y_pred_stacked = model_nb.predict(feature_matrix_valid)
accuracy_train_set = model_stacked.score(feature_matrix_train, label_matrix_train)  #get accuracy on train set
accuracy_valid_set = model_stacked.score(feature_matrix_valid, label_matrix_valid)  #get accuracy on valid set
print("Stacked ML Model, Accuracy (Train Set) : ", accuracy_train_set)
print("Stacked ML Model, Accuracy (Valid Set) : ", accuracy_valid_set)
print("Time Taken: " + str(((toc-tic)*1000)) + " ms")

<p> 
<h4>
    Classifer 8:- Random Forests (Base Estimator = Descision Tree)
</h4>
</p>

In [ ]:
tic = time.time()
#Train a Random Forest model
# Details: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
from sklearn.ensemble import RandomForestClassifier
estimators = 100
forest_depth = 25
model_rf = RandomForestClassifier(n_estimators=estimators, #The number of trees in the forest
                                  max_depth = forest_depth) #The maximum depth of the tree.
model_rf.fit(feature_matrix_train,  label_matrix_train)
toc = time.time()
y_pred_stacked = model_rf.predict(feature_matrix_valid)
accuracy_train_set = model_rf.score(feature_matrix_train, label_matrix_train)  #get accuracy on train set
accuracy_valid_set = model_rf.score(feature_matrix_valid, label_matrix_valid)  #get accuracy on valid set
print("Random Forests ML Model, Accuracy (Train Set) : ", accuracy_train_set)
print("Random Forests ML Model, Accuracy (Valid Set) : ", accuracy_valid_set)
print("Time Taken: " + str(((toc-tic)*1000)) + " ms")

<p> 
<h4>
    Classifer 9:- AdaBoostClassifier (Base Estimator = Descision Tree)
</h4>

</p>

In [ ]:
tic = time.time()
#Train a Random Forest model
# Details: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
from sklearn.ensemble import AdaBoostClassifier
estimators = 100
model_abc = AdaBoostClassifier(n_estimators=estimators) #The number of trees in the forest
model_abc.fit(feature_matrix_train,  label_matrix_train)
toc = time.time()
y_pred_stacked = model_abc.predict(feature_matrix_valid)
accuracy_train_set = model_abc.score(feature_matrix_train, label_matrix_train)  #get accuracy on train set
accuracy_valid_set = model_abc.score(feature_matrix_valid, label_matrix_valid)  #get accuracy on valid set
print("Ada Boost CLassifier ML Model, Accuracy (Train Set) : ", accuracy_train_set)
print("Ada Boost CLassifier ML Model, Accuracy (Valid Set) : ", accuracy_valid_set)
print("Time Taken: " + str(((toc-tic)*1000)) + " ms")

<p> 
<h4>
    Classifer 10:-ExtraTreesClassifier (Base Estimator = Descision Tree)
</h4>
</p>

In [ ]:
tic = time.time()
#https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html#sklearn.ensemble.ExtraTreesClassifier
from sklearn.ensemble import ExtraTreesClassifier
estimators = 200
depth = 25
model_etc = ExtraTreesClassifier(n_estimators = estimators, max_depth = depth)
model_etc.fit(feature_matrix_train,  label_matrix_train)
toc = time.time()
y_pred_stacked = model_etc.predict(feature_matrix_valid)
accuracy_train_set = model_etc.score(feature_matrix_train, label_matrix_train)  #get accuracy on train set
accuracy_valid_set = model_etc.score(feature_matrix_valid, label_matrix_valid)  #get accuracy on valid set
print("Extra Trees Classifer ML Model, Accuracy (Train Set) : ", accuracy_train_set)
print("Extra Trees Classifer ML Model, Accuracy (Valid Set) : ", accuracy_valid_set)
print("Time Taken: " + str(((toc-tic)*1000)) + " ms")

### Task 9: Plot Evaluation Metrics for Machine Learning Models

In [ ]:
from sklearn.model_selection import train_test_split #split data into train,test sets
from sklearn.metrics import plot_confusion_matrix, plot_roc_curve,  plot_precision_recall_curve
from sklearn.metrics import precision_score, recall_score
from matplotlib import pyplot as plt

In [ ]:
#make a list of classifers
ml_models_list = [model_nb, #Naive Bayes
                  model_dt, #Descision Trees
                 model_knn, #K nearest neighbors
                 model_lr, #Logistic Regression
                 model_svm, #Support Vector Machine
                  model_bc, #bagging classifer (base: descision tree)
                 model_stacked, #Stacked Model: NB, SVM, KNN, LR
                  model_rf, #random forestws
                  model_abc, #adaboost classifier
                  model_etc #extra trees classifier
                 ]

In [ ]:
models = []
accuracies = []
precisions = []
recalls = []
roc_scores = []
from sklearn.metrics import roc_auc_score
tic = time.time()
for model in ml_models_list:
    accuracy = model.score(feature_matrix_test, label_matrix_test)  #get accuracy
    y_pred = model.predict(feature_matrix_test) #get predictions
    model_name = type(model).__name__
    print("Accuracy("+ model_name + "): ", accuracy.round(4)) #display accuracy
    print("Precision("+ model_name + "): ", precision_score(label_matrix_test,y_pred).round(4)) #display precision
    print("Recall("+ model_name + "): ",recall_score(label_matrix_test,y_pred).round(4)) #display recall
    print("ROC Score("+ model_name + "): ",roc_auc_score(label_matrix_test,y_pred).round(4)) #display recall
    
    print("\n")
    models.append(model_name)
    accuracies.append(accuracy.round(4))
    precisions.append(precision_score(label_matrix_test,y_pred).round(4))
    recalls.append(recall_score(label_matrix_test,y_pred).round(4))
    roc_scores.append(roc_auc_score(label_matrix_test,y_pred).round(4))
toc = time.time()
print("Time Taken: " + str(((toc-tic)*1000)) + " ms")

In [ ]:
tic = time.time()
#Plot Confusion Matrix For all Models
fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(17,20))

for model,ax in zip(ml_models_list, axes.flatten()):
    
    plot_confusion_matrix(model, 
                          feature_matrix_test, 
                          label_matrix_test, 
                          ax=ax, 
                          cmap='Blues',
                         display_labels=["negative","positive"])
    
    ax.title.set_text(type(model).__name__)
plt.tight_layout()  
plt.show()
toc = time.time()
print("Time Taken: " + str(((toc-tic)*1000)) + " ms")

In [ ]:
tic = time.time()
#Plot ROC Curve For all Models
fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(17,30))

for model, ax in zip(ml_models_list,axes.flatten()):
    
    plot_roc_curve(model,feature_matrix_test,label_matrix_test,ax=ax)
    
    ax.title.set_text(type(model).__name__)

plt.tight_layout()  
plt.show()
toc = time.time()
print("Time Taken: " + str(((toc-tic)*1000)) + " ms")

In [ ]:
tic = time.time()
#Plot Precision Recall CUrve FOr ALl Models
fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(17,20))

for model, ax in zip(ml_models_list,axes.flatten()):
    
    plot_precision_recall_curve(model,feature_matrix_test,label_matrix_test,ax=ax)
    
    ax.title.set_text(type(model).__name__)

plt.tight_layout()  
plt.show()
toc = time.time()
print("Time Taken: " + str(((toc-tic)*1000)) + " ms")

### Task 10: Summarize Performace of Machine Learning Models

In [ ]:
results = pd.DataFrame({"Model" : models, "Accuracy" : accuracies, "Precision" : precisions, "Recall" : recalls, 'ROC Score' :roc_scores})
results

In [ ]:
results.set_index('Model', inplace=True)
results

In [ ]:
plt.figure(figsize = (17,9))
results["Accuracy"].plot(kind = 'barh')
plt.title("Comparison of Accuracies of the Models")
plt.ylabel("Models")
plt.xlim(0.70, 1.0)
plt.xlabel("Accuracy Score")
plt.show();

In [ ]:
plt.figure(figsize = (17,9))
results["Precision"].plot(kind = 'barh', color = 'red')
plt.title("Comparison of Precision of the Models")
plt.ylabel("Models")
plt.xlim(0.50, 1.0)
plt.xlabel("Precision Score")
plt.show();

In [ ]:
plt.figure(figsize = (17,9))
results["Recall"].plot(kind = 'barh', color = 'blue')
plt.title("Comparison of Recall of the Models")
plt.ylabel("Models")
plt.xlim(0.70, 1.0)
plt.xlabel("Recall Score")
plt.show();

In [ ]:
plt.figure(figsize = (17,9))
results["ROC Score"].plot(kind = 'barh', color = 'cyan')
plt.title("Comparison of ROC Score of the Models")
plt.ylabel("Models")
plt.xlim(0.70, 1.0)
plt.xlabel("ROC score")
plt.show();

In [ ]:
plt.figure(figsize = (17,9))
results.plot(kind = 'bar')
plt.title("Comparison of ROC Score of the Models")
plt.ylabel("Models")
plt.ylim(0.70, 1.0)
plt.xlabel("ROC score")
plt.show();

### Task 11: Movie Rating Generator

In [ ]:
from textblob import TextBlob

def generate_rating(ml_models,movie_review,preprocessed=False):
    
    processed_text = ""
    
    if not preprocessed:
        #pre process review
        processed_text = [process_string(movie_review)]

        print("PreProcessed Review: ")
        print(processed_text)
        print("")
        #make sure the text in string
        #processed_text = [str(processed_text)]
    else:
        processed_text = [movie_review]
    
    #generate feature matrix for the review using TF-IDF Scheme
    feature_matrix = tfidf_vectorizer.transform(processed_text)
    
    #container to store predictions
    predictions = []
    model_names = []
    
    #get predictions from all 10 classifiers
    for model in ml_models:
        
        #get model name
        model_names.append(model.__class__.__name__)
        
        #get prediction from
        predictions.append(model.predict(feature_matrix))

    
    print("Predictions By Classifiers: \n")
    for model, prediction in zip(model_names,predictions):
        print(model,":", prediction)
    
    #convert predictions from llist to numpy array
    predictions = np.array(predictions)
    
    #sum predictions from all models to get rating
    rating = np.sum(predictions)
    
    print("")
    print("Movie Rating(0-10): " + str(rating))
    
    print("")
    
    try:
        #The polarity score is a float within the range [-1.0, 1.0].
        pol =  TextBlob(movie_review).sentiment.polarity
    except:
        pol = -999999
        
    print("TextBlob Sentiment Polarity(UnPreprocessed): " + str(pol))

In [ ]:
review_test_df.info()

In [ ]:
r_num = 50
review_test_df.iloc[r_num]
#web: https://www.imdb.com/title/tt1635327/reviews?ref_=tt_urv

In [ ]:
custom_review = "I dont know how i feel about this movie. Guess i dont care https://www.imdb.com/title/tt1635327/reviews?ref_=tt_urv"
custom_review

In [ ]:
generate_rating(ml_models_list,custom_review)

In [ ]:
custom_review = "I dont know how i feel about this movie. www.google.com www.imdb.com Guess i dont care "
custom_review

In [ ]:
generate_rating(ml_models_list,custom_review)

In [ ]:
custom_review = """I have honestly never felt so confused after watching a film, a lot of people saying watch it twice but I couldn't put myself through that again. The idea is good.. but they have made it far too complicated, a bit pretentious and unfortunately no real heart to any of the characters. Disappointed."""
custom_review

In [ ]:
generate_rating(ml_models_list,custom_review)